In [2]:
%%html
<center><marquee style='width: 80%; color: blue;'><b>✌ ¡Hola a todos! Hoy vamos a trabajar un taller en clase. Muchos éxitos✌ </b></marquee><center>

<table>
    <tr>
        <td><img src="https://s3.amazonaws.com/media-p.slid.es/uploads/1485763/images/9060062/Header.png" width="300"/></td>
        <td>&nbsp;</td>
        <td>
            <h1 style="font-size:200%;color:blue;text-align:center">    <FONT COLOR="blue">   Clustering en Textos </p> Taller en clase </FONT>         </h1></td>         
        <td>
            <tp><p style="font-size:99%;text-align:center">PLN </p></tp>
            <tp><p style="font-size:115%;text-align:center">Pregrado MACC 2023-2</p></tp>
            <tp><p style="font-size:115%;text-align:center">Prof. Fabián Sánchez</p></tp>
        </td>
    </tr>
</table>

**Objetivo de la sesión**:

- Aplicar los conceptos de k-means vistos en clase al clustering de textos.

- Utilizar software Python y librerías adecuadas para el procesamiento de los datos y aplicación de los algoritmos.

- Cargar la solución antes del día **domingo 17 de septiembre a las 18:00 horas** en aulas en los grupos de trabajo.

# <FONT SIZE=5 COLOR="purple"> Punto 1. Clustering de Oraciones </FONT>

Para el siguiente ejercicio vamos a considerar el documento compuesto por unas oraciones.

a. Cargar las librerías necesesarias para el trabajo.
b. Trabaje con las *stop-words* de

```python
!pip install stop-words
from stop_words import get_stop_words
```




In [3]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [4]:
!pip install stop-words
from stop_words import get_stop_words

In [5]:
import nltk
nltk.download("all")

[nltk_data] Downloading collection 'all'
[nltk_data]    | 
[nltk_data]    | Downloading package abc to /root/nltk_data...
[nltk_data]    |   Package abc is already up-to-date!
[nltk_data]    | Downloading package alpino to /root/nltk_data...
[nltk_data]    |   Package alpino is already up-to-date!
[nltk_data]    | Downloading package averaged_perceptron_tagger to
[nltk_data]    |     /root/nltk_data...
[nltk_data]    |   Package averaged_perceptron_tagger is already up-
[nltk_data]    |       to-date!
[nltk_data]    | Downloading package averaged_perceptron_tagger_ru to
[nltk_data]    |     /root/nltk_data...
[nltk_data]    |   Package averaged_perceptron_tagger_ru is already
[nltk_data]    |       up-to-date!
[nltk_data]    | Downloading package basque_grammars to
[nltk_data]    |     /root/nltk_data...
[nltk_data]    |   Package basque_grammars is already up-to-date!
[nltk_data]    | Downloading package bcp47 to /root/nltk_data...
[nltk_data]    |   Package bcp47 is already up-to-dat

True

In [6]:
# Para vectorizar las palabras
from sklearn.feature_extraction.text import TfidfVectorizer
# Para el algoritmo k-means
from sklearn.cluster import KMeans
# Para los gráficos
import plotly.express as px
import matplotlib.pyplot as plt
# Para pandas
import pandas as pd
# Para expresiones regulares
import re
# Para peprocesamiento, selección de k en k-means
from sklearn.metrics import silhouette_samples, silhouette_score
#Para omitir los warnings
import warnings
warnings.filterwarnings("ignore")

c. Vectorizar las oraciones usando *Tf+idf* y teniendo en cuenta las anteriores palabras de parada.


In [56]:
stop_words = get_stop_words("es")

In [57]:
documents = ["los jóvenes brasileros coronados campeones del mundo",
             "La aplicación Google Translate es cada día más inteligente",
             "El reconocimiento facial de Facebook me está volviendo loco",
             "quién va a ganar el Balón de Oro este año",
             "Estas aplicaciones de cámara son divertidas",
             "Messi llevó a su equipo a ser campeón",
             "Google esta implementando inteligencia artificial para sus búsquedas",
             "La selección croata hizo una brillante campaña mundialista llegando al partido final",
             "La selección de Brasil ya no es la mejor del mundo",
             "Las extensiones de Google Chrome son útiles.",
             "Las aplicaciones de redes sociales aprovechan increíblemente la IA",
             "Instagram es una red social",
             "El Mundial de Qatar 2022 se juega en invierno"]

In [58]:
tfidf_vectorizer = TfidfVectorizer(stop_words=stop_words)

In [59]:
vectorizacion = tfidf_vectorizer.fit_transform(documents)

d. De la vectorización anterior, identifique lo siguiente:

  - los primeros 10 palabras
  - ¿Cómo están presentados los elementos de la matriz *Tf+idf*?
  - Imprima las filas 0 a 5 y columnas 0 a 10 de la matriz *Tf+iDF*.
  - ¿Qué número aparece en la posición (0,8)? ¿Qué representa este número?

In [60]:
tfidf_vectorizer.get_feature_names_out()[:10]

array(['2022', 'aplicaciones', 'aplicación', 'aprovechan', 'artificial',
       'año', 'balón', 'brasil', 'brasileros', 'brillante'], dtype=object)

Los elementos de la matriz TF-IDF (Term Frequency-Inverse Document Frequency) están presentados en forma de una matriz dispersa (sparse matrix). Una matriz dispersa es una representación eficiente en memoria que se utiliza para matrices en las que la mayoría de los elementos son ceros. En el contexto de la vectorización TF-IDF de texto, esto es especialmente útil ya que muchas palabras estarán ausentes en la mayoría de los documentos.

La matriz TF-IDF resultante de la vectorización contendrá valores numéricos en lugar de texto sin procesar. Cada fila de la matriz representa un documento de tu colección y cada columna representa una palabra o característica específica que se extrajo durante la vectorización. Cada valor en la matriz representa el peso TF-IDF de una palabra en un documento en particular.

Para acceder a los valores de la matriz TF-IDF, puedes hacerlo utilizando la notación de índices de fila y columna. Por ejemplo, `tfidf_matrix[i, j]` te daría el valor TF-IDF de la palabra en la columna `j` en el documento en la fila `i`.

In [61]:
vectorizacion[:6][:,:11].toarray()

array([[0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.45913747, 0.        ,
        0.        ],
       [0.        , 0.        , 0.42315258, 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        ],
       [0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        ],
       [0.        , 0.        , 0.        , 0.        , 0.        ,
        0.40824829, 0.40824829, 0.        , 0.        , 0.        ,
        0.        ],
       [0.        , 0.52062417, 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        ],
       [0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        ]])

El número `0.45913747` en la posición `(0, 8)` se refiere al peso TF-IDF de una palabra en el primer documento del conjunto de documentos. Si miramos los documentos, podemos identificar el primer documento:

"los jóvenes brasileros coronados campeones del mundo"

La palabra en la columna 8 se refiere a "campeones". El valor TF-IDF de `0.45913747` en la posición `(0, 8)` indica que la palabra "campeones" es relevante en el primer documento pero no es extremadamente común en el conjunto total de documentos. Esto tiene sentido, ya que en este documento en particular se habla de jóvenes brasileros que fueron coronados campeones del mundo, y "campeones" es una palabra clave en este contexto.

e. Dado que son pocos registros, utilize $k=2$ para aplicar *$k$-means*. El algoritmo de *$k$-means* tiene parámetros que se pueden cambiar. Explique para qué sirven los siguientes parámetros

```python
KMeans(n_clusters = true_k,     
       init = 'k-means++',
       max_iter = 300,
       n_init = 10)
```

1. `n_clusters` (número de clústeres o grupos):
Este parámetro `n_clusters` especifica la cantidad de clústeres en los se dividiran los datos. Particularmente, `n_clusters = true_k`, donde `true_k` es igual a 2. Esto significa que se busca dividir tus datos en 2 clústeres. De esta manera, el algoritmo intentará agrupar los datos en dos grupos distintos.

2. `init` (inicialización de centroides):
El parámetro `init` controla cómo se inicializan los centroides (los puntos representativos) de los clústeres al comienzo del algoritmo. `'k-means++'` es una opción común y recomendada. Esto significa que los centroides se inicializan de manera inteligente para acelerar la convergencia del algoritmo.

3. `max_iter` (número máximo de iteraciones):
Especifica el número máximo de iteraciones que el algoritmo *k-means* realizará antes de detenerse, incluso si no ha convergido completamente. Si los centroides no cambian significativamente entre iteraciones o el algoritmo converge antes de alcanzar este límite, se detendrá antes. Establecer un valor razonable para `max_iter` evita que el algoritmo se ejecute indefinidamente en caso de no convergencia.

4. `n_init` (número de inicializaciones aleatorias):
El algoritmo *k-means* puede ser sensible a la inicialización de centroides. Para superar esto, el parámetro `n_init` controla cuántas veces se ejecutará el algoritmo con diferentes inicializaciones aleatorias de centroides y seleccionará la mejor solución (la que tenga la suma de distancias más pequeña entre los puntos y los centroides). El valor predeterminado es 10, lo que significa que el algoritmo se ejecutará 10 veces con diferentes inicializaciones y seleccionará la mejor solución.

In [62]:
k = 2
k_means = KMeans(n_clusters=k, init = 'k-means++', max_iter = 300, n_init = 10)

f. Entrene el modelo.


In [63]:
k_means.fit(vectorizacion)

KMeans(n_clusters=2, n_init=10)

g. Identifique en cada cluster los centroides y las 5 palabras más relevantes.

In [64]:
etiquetas = k_means.labels_
centroides = k_means.cluster_centers_
feature_names = tfidf_vectorizer.get_feature_names_out()

for i in range(k):
    print(f"Cluster {i + 1}:")

    # Encuentra los índices de las 5 palabras más relevantes en el centroide del clúster
    top_word_indices = centroides[i].argsort()[-5:][::-1]

    # Obtiene las palabras más relevantes y sus valores TF-IDF
    top_words = [feature_names[idx] for idx in top_word_indices]
    top_tfidf_scores = [centroides[i][idx] for idx in top_word_indices]

    print(f"Centroide: {top_words}")
    print(f"Palabras más relevantes: {top_words}")
    print(f"Valores TF-IDF correspondientes: {top_tfidf_scores}\n")

Cluster 1:
Centroide: ['aplicaciones', 'mundo', 'selección', 'cámara', 'divertidas']
Palabras más relevantes: ['aplicaciones', 'mundo', 'selección', 'cámara', 'divertidas']
Valores TF-IDF correspondientes: [0.08804526536916774, 0.0857731814117419, 0.07534260027289114, 0.06037178474799157, 0.06037178474799157]

Cluster 2:
Centroide: ['google', 'útiles', 'chrome', 'extensiones', 'inteligencia']
Palabras más relevantes: ['google', 'útiles', 'chrome', 'extensiones', 'inteligencia']
Valores TF-IDF correspondientes: [0.36153953343392775, 0.17605466315036172, 0.17605466315036172, 0.17605466315036172, 0.1556752339858212]



h. Genere un diagrama de barras de palabras relevantes vs índice de Tf+idf.

In [65]:
for i in range(k):  # k es el número de clústeres
    print(f"\nCluster {i}:\n")

    top_word_indices = centroides[i].argsort()[-5:][::-1]
    top_words = [feature_names[idx] for idx in top_word_indices]
    top_tfidf_scores = [centroides[i][idx] for idx in top_word_indices]

    df = pd.DataFrame({'Palabras Relevantes': top_words, 'Valor TF-IDF': top_tfidf_scores})

    # Crear un gráfico de barras para cada clúster utilizando Plotly Express
    fig = px.bar(df, x='Valor TF-IDF', y='Palabras Relevantes', orientation='h',
                 title=f'Palabras Relevantes vs Valor TF-IDF (Cluster {i + 1})')

    # Mostrar el gráfico para cada clúster
    fig.show()


Cluster 0:




Cluster 1:



i. Identifique a cuál cluster pertenecerían las siguientes oraciones:

   - "cómo instalar Chrome"
   - "La final de la UCL (UEFA champions se juega este año en Madrid"


In [66]:
oraciones = ["cómo instalar Chrome", "La final de la UCL (UEFA champions se juega este año en Madrid"]

oraciones_vectorizadas = tfidf_vectorizer.transform(oraciones)

cluster_prediccion = k_means.predict(oraciones_vectorizadas)

for i, oracion in enumerate(oraciones):
    print(f"Oración: '{oracion}' pertenece al Cluster {cluster_prediccion[i]}")

Oración: 'cómo instalar Chrome' pertenece al Cluster 1
Oración: 'La final de la UCL (UEFA champions se juega este año en Madrid' pertenece al Cluster 0


# <FONT SIZE=5 COLOR="purple"> Punto 2. Clustering de Movies </FONT>

El objetivo de este punto es hacer clustering con un conjunto de datos de *overviews* de películas.

a. Importe los datos que están en e-aulas en la carpeta del taller.



In [18]:
datos = pd.read_csv('/content/drive/MyDrive/Universidad/LPN/Taller2/Movies_Dataset.csv')

b. Haga un exploración rápida de los datos.

    - Revise si hay datos faltantes
    - Explore la variable title.
    - ¿Vale la pena tener la variable *id*?



In [19]:
datos.head()

,id,title,overview
0,0,Toy Story,"Led by Woody, Andy's toys live happily in his ..."
1,1,Jumanji,When siblings Judy and Peter discover an encha...
2,2,Grumpier Old Men,A family wedding reignites the ancient feud be...
3,3,Waiting to Exhale,"Cheated on, mistreated and stepped on, the wom..."
4,4,Father of the Bride Part II,Just when George Banks has recovered from his ...


In [20]:
datos.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 45466 entries, 0 to 45465
Data columns (total 3 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   id        45466 non-null  int64 
 1   title     45460 non-null  object
 2   overview  44507 non-null  object
dtypes: int64(1), object(2)
memory usage: 1.0+ MB


In [21]:
datos = datos.dropna()
datos.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 44501 entries, 0 to 45465
Data columns (total 3 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   id        44501 non-null  int64 
 1   title     44501 non-null  object
 2   overview  44501 non-null  object
dtypes: int64(1), object(2)
memory usage: 1.4+ MB


In [22]:
datos = datos.drop(columns=['id'])
datos.head()

,title,overview
0,Toy Story,"Led by Woody, Andy's toys live happily in his ..."
1,Jumanji,When siblings Judy and Peter discover an encha...
2,Grumpier Old Men,A family wedding reignites the ancient feud be...
3,Waiting to Exhale,"Cheated on, mistreated and stepped on, the wom..."
4,Father of the Bride Part II,Just when George Banks has recovered from his ...


c. Van a realizar el ejercicio de dos formas.

    - Simplemente teniendo en cuenta las *stop_words* dentro de la función Tf+idf. (Vectorizer(stop_w...=))
    - Haciendo una normalización más completa.




In [23]:
tfidf_vectorizer = TfidfVectorizer(max_features=5000, min_df=3, stop_words=get_stop_words("en"))

In [24]:
basico = tfidf_vectorizer.fit_transform(datos['overview'])

In [25]:
def cleanPLN(texto, idioma = "es"):
  # Pasar a minusculas
  texto = texto.lower()
  # Remover caracteres especiales y tildes
  texto = re.sub(r'[á]+', 'a', texto)
  texto = re.sub(r'[é]+', 'e', texto)
  texto = re.sub(r'[í]+', 'i', texto)
  texto = re.sub(r'[ó]+', 'o', texto)
  texto = re.sub(r'[úü]+', 'u', texto)
  texto = re.sub(r'[^A-Za-z\s]+', '', texto)
  texto = texto.replace('@', '').replace('#', '')
  # Tokenizar
  tokens = nltk.word_tokenize(texto)
  # Eliminar las stop.words y Dejar palabras con más de 3 caracteres
  stop_words=get_stop_words(idioma)
  tokens_clean = [palabras for palabras in tokens if len(palabras)>3 and (palabras not in stop_words)]
  # Regrese la cadena de texto simplificada.
  texto_limpio = ' '.join(tokens_clean)
  return texto_limpio

In [26]:
datos['overview'] = datos['overview'].apply(lambda x: cleanPLN(x,"en"))

In [27]:
datos.head()

,title,overview
0,Toy Story,woody andys toys live happily room andys birth...
1,Jumanji,siblings judy peter discover enchanted board g...
2,Grumpier Old Men,family wedding reignites ancient feud nextdoor...
3,Waiting to Exhale,cheated mistreated stepped women holding breat...
4,Father of the Bride Part II,just george banks recovered daughters wedding ...


In [28]:
avanzado = tfidf_vectorizer.fit_transform(datos['overview'])

d. Utilice el método del codo y de la silueta para determinar un valor de $k$ apropiado. Si es necesario guarde la información en un archivo *.pickle*



### **Para el básico**

In [29]:
n = 20
within= []
for k in range(2,n):
  kmean = KMeans(n_clusters=k, n_init=10, random_state=0)
  kmean.fit(basico)
  within.append(kmean.inertia_)

In [30]:
fig=px.line(x=list(range(2,n)),
            y=within,
            title = "Método del Codo-Gráfica")
fig.update_xaxes(title_text = "Número de Clústers")
fig.update_yaxes(title_text = "WCSS-Inertia")
fig.show()

In [31]:
models = [KMeans(n_clusters=n, random_state=0, n_init=10).fit(basico) for n in range(2,n)]
silhouette_scores = [silhouette_score(basico, model.labels_) for model in models]

In [33]:
fig = px.line(x=list(range(2, n)), y=silhouette_scores, markers=True, line_shape='linear')
fig.update_traces(marker=dict(color='blue'), line=dict(color='blue'))
fig.update_layout(
    xaxis_title="Número de clusters $k$",
    yaxis_title="Silhouette score",
    title="Método de la Silueta",
    font=dict(size=14),
)
fig.show()

e. Entrene el modelo. Tenga en cuenta los parámetros del punto 1.


In [49]:
n = 10
kmeans = KMeans(n_clusters=n,                  ### Número de clusters
                random_state=0,                ### Semilla aleatoria.
                n_init=10)
kmeans.fit(basico)

KMeans(n_clusters=10, n_init=10, random_state=0)

f. Identifique para cada cluster sus 5 o 10 palabras más representativas. Haga diagramas de barras usando el punto 1.


In [50]:
etiquetas = kmeans.labels_
centroides = kmeans.cluster_centers_
feature_names = tfidf_vectorizer.get_feature_names_out()

for i in range(n):
    print(f"\nCluster {i}: \n")

    # Encuentra los índices de las 5 palabras más relevantes en el centroide del clúster
    top_word_indices = centroides[i].argsort()[-5:][::-1]

    # Obtiene las palabras más relevantes y sus valores TF-IDF
    top_words = [feature_names[idx] for idx in top_word_indices]
    top_tfidf_scores = [centroides[i][idx] for idx in top_word_indices]

    df = pd.DataFrame({'Palabras Relevantes': top_words, 'Valor TF-IDF': top_tfidf_scores})

    # Crear un gráfico de barras para cada clúster utilizando Plotly Express
    fig = px.bar(df, x='Valor TF-IDF', y='Palabras Relevantes', orientation='h',
                 title=f'Palabras Relevantes vs Valor TF-IDF (Cluster {i})')

    # Mostrar el gráfico para cada clúster
    fig.show()


Cluster 0: 




Cluster 1: 




Cluster 2: 




Cluster 3: 




Cluster 4: 




Cluster 5: 




Cluster 6: 




Cluster 7: 




Cluster 8: 




Cluster 9: 



### **Para el avanzado**

In [44]:
n = 20
within= []
for k in range(2,n):
    kmeanModel = KMeans(n_clusters=k).fit(avanzado)
    within.append(kmeanModel.inertia_)

In [45]:
fig=px.line(x=list(range(2,n)),
            y=within,
            title = "Método del Codo-Gráfica")
fig.update_xaxes(title_text = "Número de Clústers")
fig.update_yaxes(title_text = "WCSS-Inertia")
fig.show()

In [46]:
models = [KMeans(n_clusters=n, random_state=0).fit(avanzado) for n in range(2,n)]
silhouette_scores = [silhouette_score(avanzado, model.labels_) for model in models]

In [48]:
fig = px.line(x=list(range(2, n)), y=silhouette_scores, markers=True, line_shape='linear')
fig.update_traces(marker=dict(color='blue'), line=dict(color='blue'))
fig.update_layout(
    xaxis_title="Número de clusters $k$",
    yaxis_title="Silhouette score",
    title="Método de la Silueta",
    font=dict(size=14),
)
fig.show()

e. Entrene el modelo. Tenga en cuenta los parámetros del punto 1.

In [51]:
n = 12
kmeans = KMeans(n_clusters=n,                  ### Número de clusters
                random_state=0)                ### Semilla aleatoria.
kmeans.fit(avanzado)

KMeans(n_clusters=12, random_state=0)

f. Identifique para cada cluster sus 5 o 10 palabras más representativas. Haga diagramas de barras usando el punto 1.



In [52]:
etiquetas = kmeans.labels_
centroides = kmeans.cluster_centers_
feature_names = tfidf_vectorizer.get_feature_names_out()

for i in range(n):
    print(f"\nCluster {i}: \n")

    # Encuentra los índices de las 5 palabras más relevantes en el centroide del clúster
    top_word_indices = centroides[i].argsort()[-5:][::-1]

    # Obtiene las palabras más relevantes y sus valores TF-IDF
    top_words = [feature_names[idx] for idx in top_word_indices]
    top_tfidf_scores = [centroides[i][idx] for idx in top_word_indices]

    df = pd.DataFrame({'Palabras Relevantes': top_words, 'Valor TF-IDF': top_tfidf_scores})

    # Crear un gráfico de barras para cada clúster utilizando Plotly Express
    fig = px.bar(df, x='Valor TF-IDF', y='Palabras Relevantes', orientation='h',
                 title=f'Palabras Relevantes vs Valor TF-IDF (Cluster {i})')

    # Mostrar el gráfico para cada clúster
    fig.show()


Cluster 0: 




Cluster 1: 




Cluster 2: 




Cluster 3: 




Cluster 4: 



...


Cluster 7: 




Cluster 8: 




Cluster 9: 




Cluster 10: 




Cluster 11: 



# <FONT SIZE=5 COLOR="purple"> Punto 3. Concluya y Referencie </FONT>

- Dé algunas conclusiones del trabajo
- Incuya un par de referencias de consulta: libros, tesis, artículos, etc.